In [ ]:
%load_ext heat

In [1]:
# %%heat
# from frame import DataFrame, Series
# from index import RangeIndex, PeriodIndex
import pandas as pd
import numpy as np
# from conversions import Conversions

pdf = pd.read_csv(r"C:\Users\evanw\OneDrive\Desktop\AT\Data\GOOG.csv", index_col=0)
pdf.index = pd.to_datetime(pdf.index)


freq = "1W"

In [6]:
from Sloth import DataFrame, Series, RangeIndex, PeriodIndex

In [5]:
frame = DataFrame.from_pandas(pdf)
frame

              Open    High     Low   Close      Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900.0
2004-08-20  101.01  109.08  100.50  108.31  11428600.0
2004-08-23  110.75  113.48  109.05  109.40   9137200.0
2004-08-24  111.24  111.60  103.57  104.87   7631300.0
2004-08-25  104.96  108.00  103.88  106.00   4598900.0
...            ...     ...     ...     ...         ...
2013-02-25  802.30  808.41  790.49  790.77   2303900.0
2013-02-26  795.00  795.95  784.40  790.13   2202500.0
2013-02-27  794.80  804.75  791.11  799.78   2026100.0
2013-02-28  801.10  806.99  801.03  801.20   2265800.0
2013-03-01  797.80  807.14  796.15  806.19   2175400.0

[2148 rows x 5 columns]

In [ ]:
r = RangeIndex(start=1, stop=3, step=1)
r.start

In [ ]:
x1 = int(np.datetime64('2009-01-01').astype("datetime64[ns]"))

In [ ]:
np.datetime64('2010-01-01').astype("datetime64[ns]")

In [ ]:
x3 = np.timedelta64(12, "h").astype("timedelta64[ns]").astype("int64")

In [ ]:
np.datetime64(np.datetime64('2010-01-01'))

In [ ]:
a = np.array([1, 2, 3, 4, 5, 6])
i = np.array([1, 2])
b = np.array([["a", "b", "c", "d", "e", "f"], ["a", "b", "c", "d", "e", "f"]]).T
b

In [ ]:
pd.PeriodIndex(["2010", "2014"], freq="1Y")

In [ ]:
r.to_pandas()

In [ ]:
DataFrame(b, columns=i).loc[3:]

In [ ]:
np.asarray(frame.values_)[0:2148:1].shape

In [ ]:
%timeit frame.iloc[10:].iloc[0:3].iloc[0]

In [ ]:
%timeit pdf.iloc[10:].iloc[0:3].iloc[0]

In [ ]:
type(pdf.iloc[10:].iloc[0:3].iloc[0].values)

In [ ]:
(frame.iloc[10:].iloc[0:3].iloc[0].values == pdf.iloc[10:].iloc[0:3].iloc[0].values).all()

In [ ]:
x = pd.DataFrame(
    data=frame.values, 
#     index=frame.index.keys, 
#     columns=frame.columns.keys
)
x.index = frame.index.keys

In [ ]:
np.asarray(frame.index.keys_)[frame.index.mask]

In [ ]:
frame

In [ ]:
len(frame.values[frame.Open.values > 400])

In [ ]:
frame.values[:, frame.columns.get_item("Open")]

In [ ]:
frame.columns.get_item("Open")

In [ ]:
frame[frame.High > 800]

In [ ]:
"{} is an incorrect type".format(type(frame))

In [ ]:
DataFrame(
    values = frame.values[(frame.Open > 100).values],
    index = frame.index.keys[(frame.Open > 100).values],
    columns = frame.columns
)

In [ ]:
def boolean_array_from_slice(length, slc):
    """
    Create a boolean array corresponding to a slice.
    
    Parameters:
    length (int): Length of the array to be sliced.
    slc (slice): Slice object indicating the range.
    
    Returns:
    np.ndarray: Boolean array with True for elements in the slice range, False otherwise.
    """
    # Initialize the boolean array with all False
    bool_arr = np.zeros(length, dtype=bool)
    
    # Handle None values for slice start, stop, and step
    start = slc.start if slc.start is not None else 0
    stop = slc.stop if slc.stop is not None else length
    step = slc.step if slc.step is not None else 1
    
    # Create an array of indices that correspond to the slice
    indices = np.arange(start, stop, step)
    
    # Set the positions in the boolean array to True
    bool_arr[indices] = True
    
    return bool_arr

# Example usage:
length = 10
slc = slice(2, 8, 2)
bool_arr = boolean_array_from_slice(length, slc)
bool_arr

In [ ]:
class DataFrame_:
    def __init__(self, values, index, columns):
        self.values_ = values
        self.index = Index(index)
        self.columns = Index(columns)
        
        self.mask = slice(None, None, 4)
        
        self.iloc = IntegerLocation(self)
#         self.loc = Location(self)

    def fast_init(self, mask):
        """
        Backdoor of sorts. Allows for a quicker initialization after slicing.

        Parameters
        ----------
        mask : slice
            slice to be used
        """
        frame = self.__new__(self.__class__)
        
        frame.mask = mask
        frame.values_ = self.values_

        frame.index = self.index.fast_init(mask)
        frame.columns = self.columns

        return frame
    
    def to_pandas(self):
        return pd.DataFrame(
            self.values, 
            index=self.index.to_pandas(), 
            columns=self.columns.to_pandas()
        )
    
    def __repr__(self):
        return str(self.to_pandas())
    
    @property
    def values(self):
        return self.values_[self.mask]

In [ ]:
class Index:
    def __init__(self, index):
        # Makes the index a numpy array
        self.keys_ = np.asarray(index)
        self._initialize()
        self.reference = "object"
        
        self.mask = slice(None, None, None)
        
    def _initialize(self):
        
        length = len(self.keys_)

        # Because the index is a bunch of python objects
        # it will be stored in a python dictionary
        self.index = {}
        for i in range(length):
            self.index[self.keys_[i]] = i
    
    @property
    def keys(self):
        return np.asarray(self.keys_)[self.mask]
    
    def fast_init(self, mask: slice):
        index = self.__new__(self.__class__)

        index.index = self.index
        index.keys_ = self.keys_

        index.mask = mask

        return index
    
    def to_pandas(self):
        return pd.Index(data=self.keys)

In [ ]:
class IntegerLocation:
    def __init__(self, frame):
        self.frame = frame
        self.index = frame.index
        self.values = frame.values
    
    def __getitem__(self, arg):
        if isinstance(arg, int):
            arg = calculate_index(self.frame.mask, arg)
            return Series(self.values[arg], index=self.frame.columns.keys_, name=self.index.keys_[arg])    
        if isinstance(arg, slice):
            arg = combine_slices(self.frame.mask, arg, len(self.index.keys_))
            return self.frame.fast_init(mask=arg)

In [ ]:
DataFrame_(frame.values, frame.index.keys, frame.columns.keys).iloc[1::4]

In [ ]:
import numpy as np

def normalize_slice(s, length):
    # Handle None for start, stop, and step
    start = s.start if s.start is not None else (0 if s.step is None or s.step > 0 else length - 1)
    stop = s.stop if s.stop is not None else (length if s.step is None or s.step > 0 else -1)
    step = s.step if s.step is not None else 1
    
    # Handle negative indices
    if start < 0:
        start += length
    if stop < 0:
        stop += length
    
    return slice(start, stop, step)

def combine_slices(slice_a, slice_b, length_a):
    # Normalize slice_a and slice_b
    slice_a = normalize_slice(slice_a, length_a)
    length_b = (slice_a.stop - slice_a.start + (slice_a.step - 1)) // slice_a.step
    slice_b = normalize_slice(slice_b, length_b)
    
    # Calculate the start, stop, and step for the final slice
    start = slice_a.start + (slice_b.start * slice_a.step)
    stop = slice_a.start + (slice_b.stop * slice_a.step)
    step = slice_a.step * slice_b.step
    
    return slice(start, stop, step)

# Example usage
A = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
slice_a = slice(None, None, None)  # Slice A
slice_b = slice(1, -1, 1)          # Slice B

# First slice A to get B
B = A[slice_a]
print("B:", B)

# Then slice B to get C
C = B[slice_b]
print("C:", C)

# Now combine slices to get the final slice that directly slices A to get C
slice_combined = combine_slices(slice_a, slice_b, len(A))
C_from_A = A[slice_combined]
print("C from A:", C_from_A)


In [ ]:
class Slice:
#     cdef int start, stop, step
    
    def __init__(self, start, stop, step):
        self.start = start
        self.stop = stop
        self.step = step

In [ ]:
a[Slice(0, 4, 5)]

In [ ]:
import numpy as np

def calculate_index(slice_a, integer_b):
    # Normalize the slice to ensure it has start, stop, and step
    start = slice_a.start if slice_a.start is not None else 0
    step = slice_a.step if slice_a.step is not None else 1

    # Calculate the final index
    index = start + (integer_b * step)
    
    return index

# Example usage
A = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
slice_a = slice(2, 8, 2)  # Slice A
integer_b = 1             # Integer B

# First slice A to get B
B = A[slice_a]
print("B:", B)

# Then index B to get C
C = B[integer_b]
print("C:", C)

# Now calculate the index that directly indexes A to get C
index = calculate_index(slice_a, integer_b)
C_from_A = A[index]
print("C from A:", C_from_A)

In [ ]:
a[1:2]

In [ ]:
new_mask = np.zeros_like(a, dtype=bool)
masked_indices = np.where(mask)[0]  # Get the indices of the original array that are True in the mask
new_mask[masked_indices[slice_indices]] = True

In [ ]:
new_mask

In [ ]:
np.arange(len(a[mask]))[0:2]

In [ ]:
np.zeros_like(mask, dtype=bool)[0:2]

In [ ]:
frame.columns.keys

In [ ]:
a = np.arange(10)
a[(1, 2, 3)]

In [ ]:
%timeit pdf.iloc[:300]

In [ ]:
def floor_(dt, timeframe):
    c = Conversions()
    if timeframe == "W":
        i = (dt // c.D) * c.D
        d = i - ( ( (i - 259200000000000) % 7 ) * c.D)
    else:
        d = (dt // getattr(c, timeframe)) * getattr(c, timeframe)
    return d

In [ ]:
def ceil_(dt, timeframe):
    c = Conversions()

    if timeframe == "W":
        i = np.ceil(dt / c.D) * c.D
        d = i + ((7 - ( (i - 259200000000000) % 7 )) * c.D)
    else:
        d = np.ceil(dt / getattr(c, timeframe)) * getattr(c, timeframe)
    return (d)

In [ ]:
index = frame.index.keys_

interval = int(freq[:-1])
timeframe = freq[-1]

bins = np.arange(
    start = floor_(index[0], timeframe), 
    stop = ceil_(index[-1], timeframe), 
    step = np.timedelta64(interval, timeframe).astype("timedelta64[ns]").astype("int64")
)

split_data = np.split(frame.values, np.cumsum(
    np.bincount(np.digitize(index, bins, right=False))[1:]
))[:-1]

In [ ]:
pdf.resample(freq).mean()

In [ ]:
a = np.array([
    [1.00000e+02, 1.04060e+02, 9.59600e+01, 1.00340e+02, 2.23519e+07],
    [1.01010e+02, 1.09080e+02, 1.00500e+02, 1.08310e+02, 1.14286e+07]
])
b = np.array(["a", "2", 3, "4", 7])
b.size

In [ ]:
data = np.zeros((446, 5))

In [ ]:
np.mean(np.array([frame.resample(freq).split_data[0][0]]), axis=0)

In [ ]:
np.array([frame.resample(freq).split_data[0][0]])

In [ ]:
np.mean(frame.resample(freq).split_data[0][0].reshape(1, -1))

In [ ]:
pdf.values.shape

In [ ]:
l = len(split_data)
o = np.zeros((l, 5))
#cdef sub_array
for sub_array in range(l):
    o[sub_array] = np.mean(split_data[sub_array], axis=0)
p = [np.mean(sub_array, axis=0) for sub_array in split_data]

In [ ]:
o == frame.resample(freq).mean.values

In [ ]:
import numpy as np

def normalize_slice_py(s, length):
    start = s.start if s.start is not None else (0 if s.step is None or s.step > 0 else length - 1)
    stop = s.stop if s.stop is not None else (length if s.step is None or s.step > 0 else -1)
    step = s.step if s.step is not None else 1
    
    if start < 0:
        start += length
    if stop < 0:
        stop += length
    
    return slice(start, stop, step)

def combine_slices_py(slice_a, slice_b, length_a):
    slice_a = normalize_slice_py(slice_a, length_a)
    length_b = (slice_a.stop - slice_a.start + (slice_a.step - 1)) // slice_a.step
    slice_b = normalize_slice_py(slice_b, length_b)
    
    start = slice_a.start + (slice_b.start * slice_a.step)
    stop = slice_a.start + (slice_b.stop * slice_a.step)
    step = slice_a.step * slice_b.step
    
    return slice(start, stop, step)

In [ ]:
%load_ext cython

In [ ]:
%%cython
cdef class Slice:
    cdef int start, stop, step
    
    def __init__(self, int start, int stop, int step):
        self.start = start
        self.stop = stop
        self.step = step

cdef Slice normalize_slice(Slice s, int length):
    cdef int start, stop, step
    
    if s.start is None:
        start = 0 if s.step is None or s.step > 0 else length - 1
    else:
        start = s.start
    
    if s.stop is None:
        stop = length if s.step is None or s.step > 0 else -1
    else:
        stop = s.stop
    
    step = s.step if s.step is not None else 1
    
    if start < 0:
        start += length
    if stop < 0:
        stop += length
    
    return Slice(start, stop, step)

def combine_slices(Slice slice_a, Slice slice_b, int length_a):
    slice_a = normalize_slice(slice_a, length_a)
    length_b = (slice_a.stop - slice_a.start + (slice_a.step - 1)) // slice_a.step
    slice_b = normalize_slice(slice_b, length_b)
    
    cdef int start = slice_a.start + (slice_b.start * slice_a.step)
    cdef int stop = slice_a.start + (slice_b.stop * slice_a.step)
    cdef int step = slice_a.step * slice_b.step
    
    return slice(start, stop, step)


In [ ]:
# Setting up the data
A = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
slice_a = slice(2, 8, 2)
slice_b = slice(1, 3, 1)

# Function to use Cython combine_slices and convert to Python slice
def cython_slice():
    s = combine_slices(
        Slice(slice_a.start, slice_a.stop, slice_a.step),
        Slice(slice_b.start, slice_b.stop, slice_b.step),
        len(A)
    )
    C_from_A = A[s]
    return C_from_A

# Function to use Python combine_slices
def python_slice():
    combined_slice = combine_slices_py(slice_a, slice_b, len(A))
    C_from_A = A[combined_slice]
    return C_from_A

# Ensuring the functions return the same result
assert np.array_equal((cython_slice()), python_slice())

# Benchmarking using %timeit
%timeit cython_slice()
%timeit python_slice()


In [ ]:
def reindex_array(original, index_array):
    # Create a new array of the same size as the original, initialized with None
    new_array = [None] * len(original)
    
    # Place each element at its new index
    for original_index, new_index in enumerate(index_array):
        new_array[new_index] = original[original_index]
    
    return new_array

# Example usage
original = ['a', 'b', 'c', 'd']
index_array = [2, 0, 3, 1]
new_array = reindex_array(original, index_array)
print(new_array)


In [ ]:
column=['a','b','c','d','e']
index=['A','B','C','D','E']

x = np.random.rand(5,5)
 
# create a dataframe of random values of array
df1 = pd.DataFrame(x, columns=column, index=index)
 
print(df1)
 
print('\n\nDataframe after reindexing rows: \n', 
    df1.reindex(['B', 'D', 'A', 'C', 'E']))

In [ ]:
df2 = DataFrame(x, columns=column, index=index)

print(df2)

In [ ]:
df2.index.keys

In [ ]:
new_idx = ['B', 'D', 'A', 'C', 'E']
new_idx_idx = [df2.index.index[i] for i in new_idx]
new_idx_idx

In [ ]:
p = np.arange(4)

new_array = [None] * len(new_idx_idx)


# Place each element at its new index
for original_index, new_index in enumerate(new_idx_idx):
    print(original_index, new_index)
    new_array[new_index] = df2.index.keys[original_index]

In [ ]:
new_array

In [ ]:
help(pd)